In [33]:
import hashlib
import numpy as np
import random
import pandas as pd
import pickle as pkl

# Count min implementation

## Matrix creation
    - Creates the array of counters of width w and depth d ( TODO : Change this values and see which are better )
    - Creates array of hash functions with size d
## Update matrix
    - For the new value it implements the hashes and stores the hashcode%m in the matrix

In [2]:
def matrixCreation(w,d):
    matrix = np.zeros([d, w], dtype=np.int32)
    return matrix

In [3]:
def update(matrix,x):
    for i in range(d):
        value = int(hash_functions[i](x.encode('utf-8')).hexdigest(), 16) % 10**8
        matrix[i][value % w] += 1
        #print(value % w)

In [4]:
def queryCountMin(matrix, x):
    minimum = -1
    for i in range(d):
        value = int(hash_functions[i](x.encode('utf-8')).hexdigest(), 16) % 10**8
        if minimum == -1:
            minimum = matrix[i][value % w]
        elif matrix[i][value % w] < minimum:
            minimum = matrix[i][value % w]
    return minimum

- Maintain an matrix of (d x w) hash buckets
        - where d is the number of hash functions and and w is the range of hash values. The width w is determined by ε and the depth d is determined by δ Each bucket contains a counter.
- As new item arrives, we apply each hash function and increment the corresponding hash bucket counter. We increment d counters.
- To find the frequency count for an item, we apply the d hash functions and read the counter values from the corresponding buckets.  We end up with d  counter values. We take the minimum of the counter values and return that as the estimated count.

In [5]:
# TODO - these are not hash functions
#len(hash_functions)
def hash_function(d):
    vec = []
    vec.append(hashlib.md5)
    vec.append(hashlib.sha1)
    vec.append(hashlib.sha224)
    vec.append(hashlib.sha256)
    vec.append(hashlib.sha512)
    #for i in range(d):
     #   m = hashlib.md5()
      #  vec.append(m)
    return vec[:d]

In [6]:
CTU_NETFLOW_FILEPATH ="capture20110812.pcap.netflow.labeled"

COLUMNS = ["Date flow start","Durat","Prot","Src IP Addr:Port","Dst IP Addr:Port","Flags","Tos","Packets Bytes","Flows","Label","Labels"]

In [7]:
def determineAcc (df):
    trueValues = df.groupby(['Src IP Addr:Port']).size()
    trueValues = trueValues.reset_index()
    trueValues.columns = ['IP','True Values']
    comparisonDataFrame = trueValues
    comparisonDataFrame['countmin'] = comparisonDataFrame.apply(lambda row: queryCountMin(counterMatrix,str(row["IP"])),axis=1)

    comparisonDataFrame['countminAcc'] =  comparisonDataFrame.apply(lambda row: True if row['True Values']==row['countmin'] else False,axis=1)
    
    TP = comparisonDataFrame[comparisonDataFrame['countminAcc'] == True].count()[0]
    acc = TP / len(comparisonDataFrame)
    return (comparisonDataFrame,acc) 

In [8]:
def streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts):
    packets =[]
    with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
            #File is an Argus Unidirectional file created with ra -r so not rly csv. 
            #Let's parse by hand.
            for i,packet  in enumerate(netflow):
                #skip header
                if i == 0:
                    continue
                packet = packet.rstrip('\n')
                split_packet = packet.split("\t")
                if "->" in split_packet:
                    split_packet.remove("->")

                if "" in split_packet:
                    split_packet.remove("")

                ## Take all the ports from the ips

                if ":" in split_packet[3]:
                    split_packet[3]= split_packet[3].split(":")[0]
                if ":" in split_packet[4]:
                    split_packet[4]= split_packet[4].split(":")[0]
                
                split_packet = [e for e in split_packet if e]
                if  split_packet[3] not in hosts:
                   # print(split_packet[4])
                    update(counterMatrix,str(split_packet[3]))
                    packets.append(split_packet)
                #elif split_packet[3] not in hosts:
                #    update(counterMatrix,str(split_packet[3]))
                 #   packets.append(split_packet)

                if i == amount_of_packets:
                    break
    return packets

# Tuning of w and d

In [9]:
accuracy = 0
amount_of_packets= 1500000
hosts=["147.32.84.165","147.32.84.170","147.32.84.134","147.32.84.164","147.32.87.36","147.32.80.9","147.32.87.11"]
for w in range(100,10000,100):
        for d in range(4):
            hash_functions = hash_function(d)
            counterMatrix= matrixCreation(w,d)
            dfStream = streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts)
            df = pd.DataFrame(dfStream,columns=COLUMNS)
            _,newAccuracy = determineAcc(df)
            if newAccuracy > accuracy :
                accuracy = newAccuracy
                bestW = w 
                bestD = d 
print("BEST W")        
print(bestW)
print("BEST D")        
print(bestD)

BEST W
9900
BEST D
3


In [10]:
#0.15010490833419324

newAccuracy

0.15010490833419324

In [11]:
bestW

9900

In [12]:
bestD

3

In [13]:
# w is the best value obtained in the previous calculations
# bestW = 9900
# bestD = 3
w = bestW
d = bestD
counterMatrix= matrixCreation(w,d)
dfStream = streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts)
df = pd.DataFrame(dfStream,columns=COLUMNS)
comparisonDF, finalAccuracy = determineAcc(df)

In [77]:
len(dfStream)

1258242

In [76]:
print(finalAccuracy)

SyntaxError: invalid syntax (<ipython-input-76-b37ccccf78d6>, line 1)

In [16]:
infectedNode= "147.32.84.165"

In [17]:
comparisonDF.sort_values('True Values',ascending=False)

,IP,True Values,countmin,countminAcc
5695,147.32.84.59,173749,173749,True
5795,147.32.86.20,50390,50391,False
5740,147.32.85.25,47250,47257,False
5660,147.32.84.229,41029,41030,False
5608,147.32.84.138,36715,36721,False
5746,147.32.85.34,30512,30518,False
5634,147.32.84.171,27208,27212,False
5582,147.32.80.13,18485,18485,True
5604,147.32.84.131,17022,17024,False
5760,147.32.85.74,11716,11726,False


# Comparison between Sampling task and Sketching task
    - Compare space
    - Compare time
    - Compare accuracy

In [108]:
MinWiseSamp100 = pd.read_csv('MinWiseSample100.csv')
MinWiseSamp100 = MinWiseSamp100.drop(columns=['Unnamed: 0'])
MinWiseSamp100 = MinWiseSamp100.rename(index=str, columns={ "MinWise":"MinWise100"})


In [109]:
MinWiseSamp1000 = pd.read_csv('MinWiseSample1000.csv')
MinWiseSamp1000 = MinWiseSamp1000.drop(columns=['Unnamed: 0'])
MinWiseSamp1000 = MinWiseSamp1000.rename(index=str, columns={ "MinWise":"MinWise1000"})

In [110]:
MinWiseSamp10000 = pd.read_csv('MinWiseSample10000.csv')
MinWiseSamp10000 = MinWiseSamp10000.drop(columns=['Unnamed: 0'])
MinWiseSamp10000 = MinWiseSamp10000.rename(index=str, columns={ "MinWise":"MinWise10000"})

In [111]:
finalDF = pd.merge(comparisonDF, MinWiseSamp100,  how='left', left_on=['IP'], right_on = ['IP'])
finalDF = pd.merge(finalDF, MinWiseSamp1000,  how='left', left_on=['IP'], right_on = ['IP'])
finalDF = pd.merge(finalDF, MinWiseSamp10000,  how='left', left_on=['IP'], right_on = ['IP'])

In [112]:
finalDF.sort_values('MinWise100',ascending=False)

,IP,True Values,countmin,countminAcc,MinWise100,MinWise1000,MinWise10000
5660,147.32.84.229,41029,41030,False,11.0,164.0,1531.0
5740,147.32.85.25,47250,47257,False,6.0,37.0,358.0
5795,147.32.86.20,50390,50391,False,6.0,46.0,484.0
5608,147.32.84.138,36715,36721,False,4.0,62.0,585.0
17297,67.195.115.95,344,348,False,3.0,NaN,6.0
5791,147.32.86.190,7689,7690,False,3.0,5.0,85.0
5746,147.32.85.34,30512,30518,False,3.0,24.0,228.0
5695,147.32.84.59,173749,173749,True,2.0,35.0,337.0
5838,147.32.87.1,10072,10074,False,2.0,7.0,74.0
5760,147.32.85.74,11716,11726,False,2.0,8.0,85.0


In [113]:
finalDF= finalDF.fillna(0)

finalDF['MinWise100Values'] =finalDF['MinWise100']/100 *  len(dfStream)

finalDF['MinWise1000Values'] =finalDF['MinWise1000']/1000 *  len(dfStream)

finalDF['MinWise10000Values'] =finalDF['MinWise10000']/10000 *  len(dfStream)

In [119]:
finalDF['countminDiff'] = finalDF['countmin'] - finalDF['True Values']
finalDF['MinWise100Diff'] = abs(finalDF['MinWise100Values'] - finalDF['True Values'])
finalDF['MinWise1000Diff'] = abs(finalDF['MinWise1000Values'] - finalDF['True Values'])
finalDF['MinWise10000Diff'] = abs(finalDF['MinWise10000Values'] - finalDF['True Values'])


In [121]:
finalDF['countminErr'] = finalDF['countminDiff'] / finalDF['True Values']
finalDF['MinWise100Err'] = finalDF['MinWise100Diff'] / finalDF['True Values']
finalDF['MinWise1000Err'] = finalDF['MinWise1000Diff'] / finalDF['True Values']
finalDF['MinWise10000Err'] = finalDF['MinWise10000Diff'] / finalDF['True Values']

In [122]:
finalDF.sort_values('True Values',ascending=False)

,IP,True Values,countmin,countminAcc,MinWise100,MinWise1000,MinWise10000,MinWise100Values,MinWise1000Values,MinWise10000Values,countminDiff,MinWise100Diff,MinWise1000Diff,MinWise10000Diff,countminErr,MinWise100Err,MinWise1000Err,MinWise10000Err
5695,147.32.84.59,173749,173749,True,2.0,35.0,337.0,25164.84,44038.470,42402.7554,0,148584.16,129710.530,131346.2446,0.000000,0.855166,0.746540,0.755954
5795,147.32.86.20,50390,50391,False,6.0,46.0,484.0,75494.52,57879.132,60898.9128,1,25104.52,7489.132,10508.9128,0.000020,0.498204,0.148623,0.208552
5740,147.32.85.25,47250,47257,False,6.0,37.0,358.0,75494.52,46554.954,45045.0636,7,28244.52,695.046,2204.9364,0.000148,0.597768,0.014710,0.046665
5660,147.32.84.229,41029,41030,False,11.0,164.0,1531.0,138406.62,206351.688,192636.8502,1,97377.62,165322.688,151607.8502,0.000024,2.373385,4.029411,3.695139
5608,147.32.84.138,36715,36721,False,4.0,62.0,585.0,50329.68,78011.004,73607.1570,6,13614.68,41296.004,36892.1570,0.000163,0.370821,1.124772,1.004825
5746,147.32.85.34,30512,30518,False,3.0,24.0,228.0,37747.26,30197.808,28687.9176,6,7235.26,314.192,1824.0824,0.000197,0.237128,0.010297,0.059782
5634,147.32.84.171,27208,27212,False,0.0,12.0,142.0,0.00,15098.904,17867.0364,4,27208.00,12109.096,9340.9636,0.000147,1.000000,0.445056,0.343317
5582,147.32.80.13,18485,18485,True,1.0,9.0,116.0,12582.42,11324.178,14595.6072,0,5902.58,7160.822,3889.3928,0.000000,0.319317,0.387386,0.210408
5604,147.32.84.131,17022,17024,False,0.0,4.0,59.0,0.00,5032.968,7423.6278,2,17022.00,11989.032,9598.3722,0.000117,1.000000,0.704326,0.563880
5760,147.32.85.74,11716,11726,False,2.0,8.0,85.0,25164.84,10065.936,10695.0570,10,13448.84,1650.064,1020.9430,0.000854,1.147904,0.140839,0.087141


In [123]:
finalDF

,IP,True Values,countmin,countminAcc,MinWise100,MinWise1000,MinWise10000,MinWise100Values,MinWise1000Values,MinWise10000Values,countminDiff,MinWise100Diff,MinWise1000Diff,MinWise10000Diff,countminErr,MinWise100Err,MinWise1000Err,MinWise10000Err
0,0,2738,2741,False,0.0,2.0,27.0,0.0,2516.484,3397.2534,3,2738.0,221.516,659.2534,0.001096,1.0,0.080904,0.240779
1,0.0.0.0,1735,1735,True,0.0,0.0,17.0,0.0,0.000,2139.0114,0,1735.0,1735.000,404.0114,0.000000,1.0,1.000000,0.232860
2,1.1.1.1,62,65,False,0.0,0.0,1.0,0.0,0.000,125.8242,3,62.0,62.000,63.8242,0.048387,1.0,1.000000,1.029423
3,1.112.253.47,1,3,False,0.0,0.0,0.0,0.0,0.000,0.0000,2,1.0,1.000,1.0000,2.000000,1.0,1.000000,1.000000
4,1.114.141.158,1,2,False,0.0,0.0,0.0,0.0,0.000,0.0000,1,1.0,1.000,1.0000,1.000000,1.0,1.000000,1.000000
5,1.114.200.168,1,5,False,0.0,0.0,0.0,0.0,0.000,0.0000,4,1.0,1.000,1.0000,4.000000,1.0,1.000000,1.000000
6,1.115.20.9,1,5,False,0.0,0.0,0.0,0.0,0.000,0.0000,4,1.0,1.000,1.0000,4.000000,1.0,1.000000,1.000000
7,1.160.76.187,1,3,False,0.0,0.0,0.0,0.0,0.000,0.0000,2,1.0,1.000,1.0000,2.000000,1.0,1.000000,1.000000
8,1.161.205.7,2,8,False,0.0,0.0,0.0,0.0,0.000,0.0000,6,2.0,2.000,2.0000,3.000000,1.0,1.000000,1.000000
9,1.161.97.83,2,9,False,0.0,0.0,0.0,0.0,0.000,0.0000,7,2.0,2.000,2.0000,3.500000,1.0,1.000000,1.000000
